
## Original tSNE paper
The original tSNE paper explains the algorithm and gives a specific algorithm to optimise a projection to minimise the tSNE loss.
[L.J.P. van der Maaten and G.E. Hinton. Visualizing High-Dimensional Data Using t-SNE. Journal of Machine Learning Research 9(Nov):2579-2605, 2008.  PDF](https://lvdmaaten.github.io/publications/papers/JMLR_2008.pdf)

## Parametric tSNE paper

[L.J.P. van der Maaten. Learning a Parametric Embedding by Preserving Local Structure. In Proceedings of the Twelfth International Conference on Artificial Intelligence & Statistics (AI-STATS), JMLR W&CP 5:384-391, 2009. PDF](https://lvdmaaten.github.io/publications/papers/AISTATS_2009.pdf)


### Original notebook 
This notebook is derived from Kyle McDonald's notebook. You can find the original code here.
https://github.com/kylemcdonald/Parametric-t-SNE/blob/master/Parametric%20t-SNE%20(Keras).ipynb

In [ ]:
## Import the keras bits we need
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.objectives import categorical_crossentropy
from keras.constraints import nonneg
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras.datasets import mnist

## Outline
This notebook briefly shows how tSNE can be implemented as a straightforward loss function, which uses a precomputed "neighbourly-ness" distribution and compares a projection against that expected distribution. This is optimised to find a good layout of data in a low-d space.

This loss function can be used to find a projection of high-d data to a low-d space that preserves neighbour relations. It can be used on its own, or as a contributing part of an autoencoder. 


## Load some testing data

In [ ]:
# We work on mini batches of data
# Note that we have to form a batch_size x batch_size
# distance matrix. If batch_size is too high, we'll run out of memory
batch_size = 5000

## Load MNIST 
(x_train,y_train_labels), (x_test,y_test_labels) = mnist.load_data()
x_train = x_train.reshape(-1, 784).astype(np.float32)/255.0
x_test = x_test.reshape(-1, 784).astype(np.float32)/255.0

print x_train.shape[0], 'train samples'
print x_test.shape[0], 'test samples'

In [ ]:
# plot some random samples from the data
for i in range(64):
    plt.subplot(8,8,i+1)
    ix = np.random.randint(0, x_train.shape[0])
    plt.imshow(x_train[ix,:].reshape((28,28)), cmap="gray")
    plt.axis("off")
    

## Computing the P matrix
These function compute the probability that each pair of points $(x_i,x_j)$ should be "together", using a Gaussian kernel distance function.

$$p_{ij} = \frac{\exp( - ||x_i - x_j||^2 / 2 \sigma^2)}
{\sum_{k \neq l} \exp( - ||x_k - x_l||^2 / 2 \sigma^2)} \quad (1)$$

This is "symmetrised" to create a new variable $p^\prime_{ij}$ such that $p^\prime_{ij} = p^\prime_{ji}$, by setting
$p^\prime_{ij} = \frac{p_{ij}}{2} + \frac{p_{ji}}{2}$ **(pp. 2584)**


### Searching for $\sigma$
The width of the kernel $\sigma_i$ *for each datapoint* is adjusted by binary search so that the perplexity of the neighbourhood distribution for each point matches the a specified perplexity (perplexity = 2^entropy). 

This search adjusts $\sigma_i$ to make the distribution of $p_i$ have a fixed entropy for all $i$. The tSNE paper presents this as a kind of smooth "nearest neighbour count" -- each point $i$ should be influenced by a similar number of neighbours.


In [ ]:
def Hbeta(D, beta):
    P = np.exp(-D * beta)
    sumP = np.sum(P)
    H = np.log(sumP) + beta * np.sum(D * P) / sumP
    P = P / sumP
    return H, P

def x2p(X, u=15, tol=1e-4, print_iter=500, max_tries=50, verbose=0):
    # Initialize some variables
    n = X.shape[0]                     # number of instances
    P = np.zeros((n, n))               # empty probability matrix
    beta = np.ones(n)                  # empty precision vector
    logU = np.log(u)                   # log of perplexity (= entropy)
    
    # Compute pairwise distances
    if verbose > 0: print('Computing pairwise distances...')
    sum_X = np.sum(np.square(X), axis=1)
    # note: translating sum_X' from matlab to numpy means using reshape to add a dimension
    D = sum_X + sum_X[:,None] + -2 * X.dot(X.T)

    # Run over all datapoints
    if verbose > 0: print('Computing P-values...')
    for i in range(n):
        
        if verbose > 1 and print_iter and i % print_iter == 0:
            print('Computed P-values {} of {} datapoints...'.format(i, n))
        
        # Set minimum and maximum values for precision
        betamin = float('-inf')
        betamax = float('+inf')
        
        # Compute the Gaussian kernel and entropy for the current precision
        indices = np.concatenate((np.arange(0, i), np.arange(i + 1, n)))
        Di = D[i, indices]
        H, thisP = Hbeta(Di, beta[i])
        
        # Evaluate whether the perplexity is within tolerance
        Hdiff = H - logU
        tries = 0
        while abs(Hdiff) > tol and tries < max_tries:
            
            # If not, increase or decrease precision
            if Hdiff > 0:
                betamin = beta[i]
                if np.isinf(betamax):
                    beta[i] *= 2
                else:
                    beta[i] = (beta[i] + betamax) / 2
            else:
                betamax = beta[i]
                if np.isinf(betamin):
                    beta[i] /= 2
                else:
                    beta[i] = (beta[i] + betamin) / 2
            
            # Recompute the values
            H, thisP = Hbeta(Di, beta[i])
            Hdiff = H - logU
            tries += 1
        
        # Set the final row of P
        P[i, indices] = thisP
        
    if verbose > 0: 
        print('Mean value of sigma: {}'.format(np.mean(np.sqrt(1 / beta))))
        print('Minimum value of sigma: {}'.format(np.min(np.sqrt(1 / beta))))
        print('Maximum value of sigma: {}'.format(np.max(np.sqrt(1 / beta))))
    
    return P, beta

def compute_joint_probabilities(samples, batch_size=5000, d=2, perplexity=30, tol=1e-5, verbose=0):
    v = d - 1
    
    # Initialize some variables
    n = samples.shape[0]
    batch_size = min(batch_size, n)
    
    # Precompute joint probabilities for all batches
    if verbose > 0: print('Precomputing P-values...')
    batch_count = int(n / batch_size)
    P = np.zeros((batch_count, batch_size, batch_size))
    for i, start in enumerate(range(0, n - batch_size + 1, batch_size)):   
        curX = samples[start:start+batch_size]                   # select batch
        P[i], beta = x2p(curX, perplexity, tol, verbose=verbose) # compute affinities using fixed perplexity
        P[i][np.isnan(P[i])] = 0                                 # make sure we don't have NaN's
        P[i] = (P[i] + P[i].T) # / 2                             # make symmetric
        P[i] = P[i] / P[i].sum()                                 # obtain estimation of joint probabilities
        P[i] = np.maximum(P[i], np.finfo(P[i].dtype).eps)

    return P

## Getting P
Now we actually run the code and compute the P matrix for our data. The output is an `n x batch_size x batch_size` matrix, where `n` is the number of batches. 

I shuffled the data to make sure each minibatch has similar distribution.

### Perplexity
The perplexity is the key parameter that controls the tSNE layout. Values between 15 and 80 are usually suitable. The paper claims that tSNE is mostly invariant to the setting of this parameter, but I've found that is relatively important (but fairly smooth: 13 will look like 14).

Perplexity is:
$$Perp(P_i) = 2^{H(P_i}),$$ and the $H(P_i)$ is the Shannon entropy:
$$H(P_i)=-\sum_j p_{ji} log_2(p_{ji}).$$

So we optimise $\sigma_i$ to give a similar $H(P_i)$ for each point.


In [ ]:
# Compute P matrix
# this can take a little while
np.random.shuffle(x_train)
P = compute_joint_probabilities(x_train, batch_size=5000, perplexity=60, verbose=2)

## Plotting the P matrix
We can see what the P matrix looks like. We just plot a subsection of the first minibatch P matrix, because the whole set of P matrices is huge. The result won't be very interesting, because the data order is shuffled.

In [ ]:
plt.figure(figsize=(8,8))
# show the log matrix
P0 = np.array(P[0][:500, :500])
plt.imshow(np.log(P0), interpolation="nearest", cmap="viridis")

# tSNE loss
To optimise our projection, we use the tSNE loss as an objective function. 

This loss is given by the Kullback-Leibler divergence between the high-dimensional neighbourhood distribution (P) and the a low-dimensional neighbourhood distribution (Q). The Q distribution uses a t-distributed neighbourhood function instead of a Gaussian to account for the distortion of space in down-projecting (hence **t**SNE).

$$ q_{ij} = \frac{(1+||y_i - y_j||^2)^{-1}}{\sum_{k\neq l}(1+||y_k + y_l||^2)^{-1}} \quad (4)$$ 

$$C=KL(P||Q)=\sum_i\sum_j p_{ij} \log \frac{p_{ij}}{q_{ij}}$$

This is differentiable, and Theano works out the derivatives for us.

`tsne(P,x)` takes the precomputed P matrix and a set of projected values (for one minibatch) and returns the loss.



## The t-distribution
The use of the t-distribution seems to be very important in getting good results in the projection. The heavy tails allow the low-dimensional neighbourhood to "stretch" much further.

In [ ]:
## Show the difference between the T and Gaussian distributions
x = np.linspace(-5,5, 200)
gauss = np.exp(-x**2)
t =  (1+x**2)**(-1)
plt.plot(x,gauss, lw=2)
plt.plot(x,t, '--', lw=2)
plt.fill_between(x, gauss, t, hatch='//', facecolor='none', alpha=0.2)
plt.legend(["Gaussian", "t distribution $\\nu=1$"])
plt.figure()

plt.semilogy(x,gauss, lw=2)
plt.semilogy(x,t, '--', lw=2)
plt.fill_between(x, gauss, t, hatch='//', facecolor='none', alpha=0.2)
plt.legend(["Gaussian", "t distribution $\\nu=1$"])


## Loss function

In [ ]:
# P is the joint probabilities for this batch (Keras loss functions call this y_true)
# activations is the low-dimensional output (Keras loss functions call this y_pred)

def tsne(P, activations):
#     d = K.shape(activations)[1]
    d = 2 # TODO: should set this automatically, but the above is very slow for some reason
    n = 5000 #batch_size # TODO: should set this automatically
    v = d - 1.
    eps = K.variable(10e-15) # needs to be at least 10e-8 to get anything after Q /= K.sum(Q)
    sum_act = K.sum(K.square(activations), axis=1)
    Q = K.reshape(sum_act, [-1, 1]) + -2 * K.dot(activations, K.transpose(activations))
    Q = (sum_act + Q) / v
    Q = K.pow(1 + Q, -(v + 1) / 2)
    Q *= K.variable(1 - np.eye(n))
    Q /= K.sum(Q)
    Q = K.maximum(Q, eps)
    C = K.log((P + eps) / (Q + eps))
    C = K.sum(P * C)
    return C

## Straight tSNE
We can directly optimise the tSNE using a MLP structure. We map inputs to a two-dimensional space, and then backprop to update weights to minimise the tSNE loss.


In [ ]:
# Simple one-hidden-layer MLP
# 784 - 256 - 2 (tsne) 

input_img = Input(shape=(x_train.shape[1],))
l = input_img
# hidden layer
l = Dense(256,  activation='relu')(l)
# output layer
l = Dense(2, activation='linear')(l)

# note that tSNE gets the P matrix from the targets. 
tsne_model = Model(input=input_img, output=l)
sgd = SGD(lr=0.1)
# Use stochastic gradient descent
tsne_model.compile(loss=tsne, optimizer=sgd)

## Model fitting
The only subtlety is that the target is the P matrix -- that is what the tsne loss compares the outputs to, via the Q matrix which it forms internally.

In [ ]:
# Outputs are from the P matrix
y_train = P.reshape(x_train.shape[0], -1)
print(x_train.shape)
print(y_train.shape)
# must not shuffle or we might mix rows from different P matrices!
tsne_model.fit(x_train, y_train, batch_size=5000, shuffle=False, nb_epoch=100)

## Plot the results
The output can now be found via `.predict()` Note that because we have learned a **parametric model**, we can project new, unseen data onto this space if we want to. We haven't just learned where the new points go, but the transformation that takes them there.

In [ ]:
plt.figure()
embedding = tsne_model.predict(x_train)
plt.scatter(embedding[:,0], embedding[:,1], marker='o', s=1, edgecolor='', c=y_train_labels, cmap="viridis")
plt.figure()
embedding = tsne_model.predict(x_test)
plt.scatter(embedding[:,0], embedding[:,1], marker='o', s=1, edgecolor='', c=y_test_labels, cmap="viridis")

## A tSNE autoencoder
We can use the tSNE loss as a contributing loss function in any model. We just need to have a layer with the right dimension which we can compute a Q matrix from.
### Autoencoder
We can show this by building an autoencoder, and optimising the loss on both the reconstruction error (how well can the autoencoder reconstruct the input after going through the latent variable bottleneck), **and** the tSNE loss (how similar is the neighbourhood distribution of the high dimensional space to the neighoburhood distribution of the latent variable bottleneck)

### Inversion
Note that we could use to **invert** the transform, since the layers from the latent space to the reconstructed space reconstruct a high-d point given a low-d point.


In [ ]:
from keras.constraints import nonneg
from keras.layers import Input, Dense, Lambda
from keras.models import Model

# Deep autoencoder, with the latent layer subject to the tSNE penalty
# 784 - 256 - 128 - 16 - 2 (tsne) - 16 - 128 - 256 - 784

input_img = Input(shape=(x_train.shape[1],))
l = input_img
l = Dense(256,  activation='relu')(l)
l = Dense(128,  activation='relu')(l)
l = Dense(16, activation='relu')(l)
# this is the latent variable bottleneck layer (here, with D=2)
l = Dense(2)(l)

# note that we keep a hold of the projected layer; we apply the tSNE loss
# to this output layer
proj_layer = l

l2=Dense(16, activation='relu')(l)
l2=Dense(128,  activation='relu')(l2)
l2=Dense(256,  activation='relu')(l2)
l2=Dense(784, activation='linear')(l2)
# the last layer should be the reconstructed values

out_layer = l2

# we specify *two* outputs for this model
auto_model = Model(input=input_img, output=[out_layer, proj_layer])

# we also create the inverse model
latent_input = Input(shape=(2,))
decoder_layers = auto_model.layers[-4:]

# surely this can't be the right way of doing this?!
decoder = reduce(lambda x,y:y(x), decoder_layers, latent_input)
inverse_model = Model(input=latent_input, output=decoder)

sgd = SGD(lr=0.1)
# and specify dual loss functions: mse + tsne
auto_model.compile(loss=['mean_squared_error',tsne], optimizer=sgd)

In [ ]:
# note the two outputs: first one is autoencoder reconstruction, second is the P matrix
auto_model.fit(x_train, [x_train, y_train], batch_size=5000, shuffle=False, nb_epoch=100)

In [ ]:
plt.figure()
# use the second output -- the latent variables
embedding = auto_model.predict(x_train)[1]
plt.scatter(embedding[:,0], embedding[:,1], marker='o', s=1, edgecolor='', c=y_train_labels, cmap="viridis")
plt.figure()
embedding = auto_model.predict(x_test)[1]
plt.scatter(embedding[:,0], embedding[:,1], marker='o', s=1, edgecolor='', c=y_test_labels, cmap="viridis")

## Test the inversion
We can try randomly generating a point near an existing point and see what we get as a reconstruction:

In [ ]:

# use the second output -- the latent variables
embedding = auto_model.predict(x_train)[1]

In [ ]:
import random
# choose a random point somewhere on a line between two existing points
def random_inverse_digit(embedding, inverse_model):
    n1, n2 = random.choice(embedding), random.choice(embedding)
    t = np.random.uniform(0,1)
    n = t*n1 + (1-t)*n2
    digit = inverse_model.predict([n[None,:]])
    print digit.shape
    plt.imshow(digit.reshape(28,28), cmap="gray", interpolation="nearest")
    
random_inverse_digit(embedding, inverse_model)

## A deep autoencoder
We can make any kind of autoencoder structure, and keep the tSNE loss. This model is a deep denoising autoencoder.

In [ ]:
from keras.constraints import nonneg
from keras.layers import Input, Dense, Lambda
from keras.layers.noise import GaussianNoise
from keras.models import Model

# Deep autoencoder, with the latent layer subject to the tSNE penalty
# 784 - 256 - 128 - 16 - 2 (tsne) - 16 - 128 - 256 - 784

# std. dev. of noise added during training
sigma = 0.25

input_img = Input(shape=(x_train.shape[1],))
l = input_img
l = GaussianNoise(sigma)(l)
l = Dense(256,  activation='relu')(l)
l = Dense(128,  activation='relu')(l)
l = Dense(16, activation='relu')(l)
# this is the latent variable bottleneck layer (here, with D=2)
l = Dense(2)(l)

# note that we keep a hold of the projected layer; we apply the tSNE loss
# to this output layer
proj_layer = l

l=Dense(16, activation='relu')(l)
l=Dense(128,  activation='relu')(l)
l=Dense(256,  activation='relu')(l)
l=Dense(784, activation='linear')(l)
# the last layer should be the reconstructed values

out_layer = l

# we specify *two* outputs for this model
auto_model = Model(input=input_img, output=[out_layer, proj_layer])

# we also create the inverse model
latent_input = Input(shape=(2,))
# surely this can't be the right way of doing this?!
decoder = reduce(lambda x,y:y(x), decoder_layers, latent_input)
inverse_model = Model(input=latent_input, output=decoder)

sgd = SGD(lr=0.1)
# and specify dual loss functions: mse + tsne
auto_model.compile(loss=['mean_squared_error',tsne], optimizer=sgd)

In [ ]:
## Test the deep denoising autoencoder
plt.figure()
# use the second output -- the latent variables
embedding = auto_model.predict(x_train)[1]
plt.scatter(embedding[:,0], embedding[:,1], marker='o', s=1, edgecolor='', c=y_train_labels, cmap="viridis")
plt.figure()
embedding = auto_model.predict(x_test)[1]
plt.scatter(embedding[:,0], embedding[:,1], marker='o', s=1, edgecolor='', c=y_test_labels, cmap="viridis")


plt.figure()
# test inversion
random_inverse_digit(embedding, inverse_model)